Makes reconstruction predictions and visualize the predicted meshes. Trained models are stored as Kaggle datasets. We were able to see the visualization results in the output logs when we commit the notebook and it is completed

In [1]:
!pip install K3D>=2.9.4

In [2]:
import sys
sys.path.append('../input/rbirkelandmvcnnadopted/MVCNN-PyTorch')
import k3d
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import torchvision.transforms as transforms

import argparse
import numpy as np
import time
import os

from models.resnet import *
from models.mvcnn import *
import util
from logger import Logger
from custom_dataset import MultiViewDataSet

In [3]:
transform = transforms.Compose([
    transforms.CenterCrop(500),
    transforms.Resize(224),
    transforms.ToTensor(),
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
dset_val = MultiViewDataSet('../input/modelnetdataset/shaded_images/modelnet40_images_new_12x',
                            '../input/modelnetvoxels0608/model_net_voxel_indices', 'test', 
                            transform=transform)
val_loader = DataLoader(dset_val, batch_size=1, shuffle=True, num_workers=2)

classes = dset_val.classes
print(len(classes), classes)

40 ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']


In [5]:
model = resnet18(pretrained=True, num_classes=len(classes), attention=False, view_dropout=False, reconstruction=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [6]:
trained_model_path = '../input/ml3dprojectmodels/checkpoint/resnet18_checkpoint.pth.tar'
def load_checkpoint():
    global best_acc, start_epoch
    # Load checkpoint.
    print('\n==> Loading checkpoint..')
    assert os.path.isfile(trained_model_path), 'Error: no checkpoint file found!'

    checkpoint = torch.load(trained_model_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])

In [7]:
load_checkpoint()


==> Loading checkpoint..


In [8]:
model.to(device)
cudnn.benchmark = True

In [9]:
def visualize_pointcloud(point_cloud, point_size, flip_axes=False, name='point_cloud'):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    if flip_axes:
        point_cloud[:, 2] = point_cloud[:, 2] * -1
        point_cloud[:, [0, 1, 2]] = point_cloud[:, [0, 2, 1]]
    plt_points = k3d.points(positions=point_cloud.astype(np.float32), point_size=point_size, color=0xd0d0d0)
    plot += plt_points
    plt_points.shader = '3d'
    plot.display()

In [10]:
for k in range(10):
    for i, (images, gt_labels, gt_voxels) in enumerate(val_loader):
        images = np.stack(images, axis=1)

        images = torch.from_numpy(images)

        images, gt_labels, gt_voxels = images.cuda(device), gt_labels.cuda(device), gt_voxels.cuda(device)
        images, gt_labels, gt_voxels = Variable(images), Variable(gt_labels), Variable(gt_voxels)

        # compute output
        outputs = model(images)
        print("Output shape:", outputs.shape)
        np_outputs = torch.exp(outputs).detach().cpu().numpy()
        pred_s = np.squeeze(np_outputs[:, 1, :, :, :])
        preds_occupy = pred_s >= 0.4
        point_list = np.concatenate([c[:, np.newaxis] for c in np.where(preds_occupy)], axis=1)
        visualize_pointcloud(point_list, 1, flip_axes=False, name='occupancy_grid')
        gt_voxels = gt_voxels.detach().cpu().numpy()
        pred_s_gt_voxels = gt_voxels[0] #np.squeeze(gt_voxels[: , :, :, :])
        point_list_gt_voxels = np.concatenate([c[:, np.newaxis] for c in np.where(pred_s_gt_voxels)], axis=1)
        print(point_list_gt_voxels.shape)
        visualize_pointcloud(point_list_gt_voxels, 1, flip_axes=False, name='occupancy_grid')
        print(gt_voxels.shape)
        break

Output shape: torch.Size([1, 2, 34, 34, 34])


Output()

(1416, 3)


Output()

(1, 34, 34, 34)
Output shape: torch.Size([1, 2, 34, 34, 34])


...

(1, 34, 34, 34)
Output shape: torch.Size([1, 2, 34, 34, 34])


Output()

(3510, 3)


Output()

(1, 34, 34, 34)
